## Functions and basic types

We will start with functions and basic types.  A function is declared with `def` followed by a name, and then a
parenthesis with 0 or more parameters.

Python functions has several special abilities with functions:

- Can take default values for parameters
- Can specify keyword at the calling of a function
- Can take positional only parameters using `/`
- Can take keyword only parameters using `*`
- Can pass in a list for positional args with the `*indentifier` (typically `*args`) operator
    - If specified, must come before the `*` marker
- Can pass in a dictionary for keyword args using the `**identifier` (typically `**kwargs`) operator
    - if `*args` is used, must come after it

All these rules can make it pretty complicated pretty fast.  I recommend sticking to a subset of these.  It is often
useful to specify by kw-only even if more verbose.  But to show you how complicated it can get here are some examples

## Simple function

Here is a basic function with type annotations.  It takes two arguments, one is named `name` and takes a `str` type.
The second parameter is called `age` and takes an `int` type.  It returns a `str` denoted by `-> str`

Note that in python, types are purely optional and are calculated by a static type checker.  But at runtime, the python
interpreter does not use any of this information.  The annotations are available with reflection though.

However, you should _always_ use types, even if they optional.

### Our first real function

We are going to start building an http REST client.  We will start with a very basic function, and slowly build it up with new parameters.  As an aside, python distinguishes parameters vs. arguments.  Parameters are the _declaration_ of the types in the function, and arguments are the values passed in to the function when it is called.

## A simple service

In order to demonstrate all the different function parameter declarations, we will build a tiny http service, and a CLI
client.  For now, there is no need to understand how the fastapi service works.  Over time, some of the features like
the decorator being used will be explained.

Due to the way that the fastapi works, this code can not be run in the notebook and it lives in a file service.py in
the same folder as this notebook

```python
from fastapi import FastAPI
from datetime import datetime

app = FastAPI()


@app.get("/time")
def date_now():
    return {"time": datetime.now().isoformat() }
```

In a shell, run the following

```sh
uvicorn service:app --reload
```

In [6]:
import requests

def basic_get():
    """Returns name is age"""
    return requests.get(
        "http://127.0.0.1:8000/time"
    )

value = basic_get()
print(value.json())

{'time': '2023-08-21T07:40:41.745046'}
